# Columbus Neighborhoods and Houses Assessment

# 1. Introduction/Business Problem

<font size="3">

__Business Problem__: Assess neighborhoods of selected houses in Columbus Ohio by looking at nearby venues, schools and crimes, and use the assessment to help making decision of purchsing a house.

__Background__: The audiences whom this project aims at are real estate buyers and sellers. When we move in a house or apartment, we live not only in a building, but also in a neighborhood. We not only care about conditions and price of a house, but also safety, convenience, and good schools for kids. We need to evaluate the neighborhood of selected houses based on nearby venues, schools and crimes. Based on clients' preference, I design a ranking system and evaluate each neighborhood. I return a report to help clients to have a better assessment of the houses' environment and narrow down options.
<font>

## 2. Data

### Data description:

<font size="3">

__Houses Information__: chose 10 houses from Realtor.com with the following filters: Location: Columbus OH, Price: from \\$180,000 to \\$220,000, Bedroom 3+, Bathroom 2+, Property Type: Single Family. The data is stored in RealEstateInfo.csv.

__Nearby Venues__: explore nearby venues within a mile of each house by __Foursquare API__. We obtain counts of different categories of venues by wrangling the raw data.

__School Information__: obtain evaluation of public schools (including district and charter) within 5 miles from each house from greatschools.org. GreatSchools’ Summary Rating is used to evaluate each school. The Top 1 school's score and Top 3 schools' average scores for elementary, middle and high schools respectively are listed in SchoolInfo.csv.

__Crimes Information__: obtain CrimeRisk™ Score and Probability of a Crime Claim from https://locationinc.com/claim-probability-tools/residential-crime-claim/ and store the data in CrimesInfo.csv.
<font>

In [1]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [2]:
#foursquare account
CLIENT_ID = "4UFZAFUMNFADWCHWO2WWR1NJFL2Q1044QF2BEDOQXFJGW4DR"
CLIENT_SECRET =  # your Foursquare Secret
access_token = 
VERSION = '20200624' # Foursquare API version

In [14]:
#get json file of venues for the given postal code with the given location dataframe df_loc
def VenuesJson(latitude, longitude):
    #parameters
    LIMIT = 500
    radius = 1600
     # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                latitude, 
                longitude, 
                radius, 
                LIMIT)
    return requests.get(url).json()

#get category names from json file
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
#get categories from venues in json format
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
#get category counts from json data, return a series with category type and percentage
def get_category_counts(venues):
    nearby_venues = pd.json_normalize(venues) # flatten JSON
    s_temp = nearby_venues.apply(get_category_type, axis=1)
    s_temp2 = s_temp.map(convert)
    s_temp2 = s_temp2.replace(np.nan, 'OTHER')
    return  s_temp2.value_counts()

In [5]:
#wrangling housing date and add coordinates
df_house = pd.read_csv('RealEstateInfo.csv', thousands=',', encoding= 'unicode_escape')

#remove '\xa0'
for i in range(df_house.shape[0]):
    df_house.loc[i, 'ADDRESS'] = df_house.loc[i, 'ADDRESS'].replace('\xa0', ' ')
    
#add coordinates
geolocator = Nominatim(user_agent="cbus_explorer")
for i in range(df_house.shape[0]):
    address = df_house.loc[i, 'ADDRESS']
    location = geolocator.geocode(address)
    df_house.loc[i, 'LATITUDE'] = location.latitude
    df_house.loc[i, 'LONGITUDE'] = location.longitude    

#save the data into a csv file name RealEstateInfo(processed).csv
df_house.to_csv('RealEstateInfo(processed).csv', index = False)

In [6]:
df_house = pd.read_csv('RealEstateInfo(processed).csv')
df_house

,ADDRESS,BED,BATH,INNER_DIM,OUTER_DIM,PRICE,LATITUDE,LONGITUDE
0,"6039 Rover Ln, Columbus, OH 43232",3,2.5,1682,6098.0,185000,39.918816,-82.836154
1,"4416 Wingfield St, Columbus, OH 43231",3,2.0,1400,7405.0,180000,40.063049,-82.928520
2,"1292 Eagle View Dr, Columbus, OH 43228",3,2.0,1209,7405.0,210000,39.926817,-83.109923
3,"3785 Sunbury Rd, Columbus, OH 43219",3,2.0,1237,35719.2,214500,40.036869,-82.931113
4,"4270 Umiak Dr, Columbus, OH 43207",3,2.5,1936,7405.0,199999,39.879087,-82.966949
5,"3322 Pine Valley Rd, Columbus, OH 43219",3,3.5,1742,7405.0,184900,40.012570,-82.920537
6,"5029 Kilbourne Run Pl, Columbus, OH 43229",3,2.5,1498,10890.0,199900,40.072652,-82.953729
7,"936 Cypress Ridge Pl, Columbus, OH 43228",3,2.5,1476,24393.6,209900,39.931910,-83.129554
8,"2651 Winningwillow Dr, Columbus, OH 43207",3,2.5,1521,6970.0,195000,39.892353,-82.927716
9,"1392 Bryson Rd, Columbus, OH 43224",4,3.0,2066,8712.0,200000,40.053396,-82.980858


In [7]:
#visualize all housing location by using folium
import folium
print(folium.__version__)

#coordinates of Columbus
address = 'Columbus, Ohio, USA'

location = geolocator.geocode(address)
c_lat = location.latitude
c_long = location.longitude

# create map and display it
columbus_map = folium.Map(location=[c_lat, c_long], zoom_start=10)

# loop through the 100 crimes and add each to the map
for lat, lng, label in zip(df_house.LATITUDE, df_house.LONGITUDE, df_house.ADDRESS):
    folium.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='yellow',
        fill=True,
        popup=label,
        par_html = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(columbus_map)

columbus_map

0.11.0


In [9]:
#a list of dictionaries containing address and venues
features = []

for i in range(df_house.shape[0]):
    #coordinates
    latitude = df_house.LATITUDE[i]
    longitude = df_house.LONGITUDE[i]
    results = VenuesJson(latitude, longitude)
    try:
        venues = results['response']['groups'][0]['items']
    except:
        print(i)
    features.append({'address':df_house.ADDRESS[i], 'venues':venues})

In [10]:
import json

#save the venues data in a json file
json_dict = {'features': features}
with open('NearbyVenues.json', 'w') as json_file:
    json.dump(json_dict, json_file)

In [11]:
#read json file of venues data
import json

with open('NearbyVenues.json') as f:
    venues_json = json.load(f)

In [12]:
import ast

#dictionary of converting categories to 'parent' categories to reduce the number of categories
f = open('catogeries_convert_list.txt', 'r')
convert = f.read()
convert = ast. literal_eval(convert)
f.close()

In [15]:
#df_cat is a dataframe consists of categories and their counts
df_cat = pd.DataFrame(columns = ['categories'])

#process data from venues_json
for i in range(len(venues_json['features'])):
    venues = venues_json['features'][i]['venues']
    df_temp = pd.DataFrame(get_category_counts(venues), columns = [i]).reset_index()
    df_temp.rename(columns = {'index':'categories'}, inplace = True)
    #update df_cat by outer join with df_temp
    df_cat = df_cat.merge(df_temp, how = 'outer', on = 'categories')
    
df_cat.replace(np.nan, 0, inplace = True)
df_cat = df_cat.set_index('categories')
df_cat.index.name = None
df_cat = df_cat.T

df_cat = df_house[['ADDRESS', 'LATITUDE', 'LONGITUDE']].merge(df_cat, left_index = True, right_index = True)
df_cat.to_csv('NearbyVenuesByCategories.csv', index = False)
df_cat

,ADDRESS,LATITUDE,LONGITUDE,FOOD,OTHER,SHOPPING,GROCERY/MARKET,GAS_STATION,CONVENIENCE/PHARMACY,RECREATION,HOME_SUPPLY,AUTO,BAR,PET,BANK/ATM,BEAUTY
0,"6039 Rover Ln, Columbus, OH 43232",39.918816,-82.836154,16.0,11.0,4.0,3.0,3.0,3.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0
1,"4416 Wingfield St, Columbus, OH 43231",40.063049,-82.928520,14.0,15.0,22.0,0.0,1.0,2.0,5.0,5.0,1.0,0.0,1.0,1.0,0.0
2,"1292 Eagle View Dr, Columbus, OH 43228",39.926817,-83.109923,30.0,12.0,1.0,1.0,4.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0
3,"3785 Sunbury Rd, Columbus, OH 43219",40.036869,-82.931113,0.0,4.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,1.0
4,"4270 Umiak Dr, Columbus, OH 43207",39.879087,-82.966949,2.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,"3322 Pine Valley Rd, Columbus, OH 43219",40.012570,-82.920537,7.0,6.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"5029 Kilbourne Run Pl, Columbus, OH 43229",40.072652,-82.953729,21.0,13.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,0.0,4.0,0.0
7,"936 Cypress Ridge Pl, Columbus, OH 43228",39.931910,-83.129554,27.0,14.0,1.0,1.0,4.0,3.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0
8,"2651 Winningwillow Dr, Columbus, OH 43207",39.892353,-82.927716,1.0,6.0,0.0,0.0,1.0,1.0,9.0,1.0,1.0,1.0,0.0,0.0,0.0
9,"1392 Bryson Rd, Columbus, OH 43224",40.053396,-82.980858,21.0,11.0,1.0,3.0,3.0,3.0,1.0,1.0,1.0,0.0,1.0,4.0,0.0


In [16]:
df_school = pd.read_csv('SchoolInfo.csv')
df_school

,ADDRESS,LATITUDE,LONGITUDE,PUB_E_TOP1_SCORE,PUB_E_TOP3_AVE_SCORE,PUB_M_TOP1_SCORE,PUB_M_TOP3_AVE_SCORE,PUB_H_TOP1_SCORE,PUB_H_TOP3_AVE_SCORE,PUB_E_NUM,PUB_M_NUM,PUB_H_NUM
0,"6039 Rover Ln, Columbus, OH 43232",39.918816,-82.836154,8,7.00,7,6.67,7,6.00,38,22,17
1,"4416 Wingfield St, Columbus, OH 43231",40.063049,-82.928520,9,8.00,8,7.33,8,6.33,57,22,19
2,"1292 Eagle View Dr, Columbus, OH 43228",39.926817,-83.109923,9,7.33,9,7.67,9,6.33,36,19,12
3,"3785 Sunbury Rd, Columbus, OH 43219",40.036869,-82.931113,8,8.00,8,7.67,8,6.00,59,25,22
4,"4270 Umiak Dr, Columbus, OH 43207",39.879087,-82.966949,9,8.00,5,5.00,6,5.00,21,13,11
5,"3322 Pine Valley Rd, Columbus, OH 43219",40.012570,-82.920537,8,7.67,8,7.67,8,7.00,56,29,24
6,"5029 Kilbourne Run Pl, Columbus, OH 43229",40.072652,-82.953729,9,8.33,8,7.33,8,6.67,58,24,19
7,"936 Cypress Ridge Pl, Columbus, OH 43228",39.931910,-83.129554,9,7.33,9,7.67,9,5.33,34,18,10
8,"2651 Winningwillow Dr, Columbus, OH 43207",39.892353,-82.927716,9,7.00,7,6.00,7,6.67,41,23,21
9,"1392 Bryson Rd, Columbus, OH 43224",40.053396,-82.980858,9,8.33,7,7.00,9,8.00,59,28,26


In [17]:
df_crime = pd.read_csv('CrimesInfo.csv')
df_crime

,ADDRESS,LATITUDE,LONGITUDE,CRIME_RISK_SCORE,PROBABILITY_CLAIM
0,"6039 Rover Ln, Columbus, OH 43232",39.918816,-82.836154,216,0.066667
1,"4416 Wingfield St, Columbus, OH 43231",40.063049,-82.928520,201,0.058824
2,"1292 Eagle View Dr, Columbus, OH 43228",39.926817,-83.109923,300,0.076923
3,"3785 Sunbury Rd, Columbus, OH 43219",40.036869,-82.931113,217,0.066667
4,"4270 Umiak Dr, Columbus, OH 43207",39.879087,-82.966949,160,0.050000
5,"3322 Pine Valley Rd, Columbus, OH 43219",40.012570,-82.920537,228,0.066667
6,"5029 Kilbourne Run Pl, Columbus, OH 43229",40.072652,-82.953729,168,0.052632
7,"936 Cypress Ridge Pl, Columbus, OH 43228",39.931910,-83.129554,216,0.066667
8,"2651 Winningwillow Dr, Columbus, OH 43207",39.892353,-82.927716,221,0.066667
9,"1392 Bryson Rd, Columbus, OH 43224",40.053396,-82.980858,193,0.058824
